### Install requirements

First, run the cells below to install the requirements:

In [1]:
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
import huggingface_hub

In [4]:
huggingface_hub.utils.enable_progress_bars

<function huggingface_hub.utils.tqdm.enable_progress_bars() -> None>

In [5]:
import os
# from pprint import pprint
# import json

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [6]:
model_name = "/home/asif/llm/llama-2-7b-model"
finetuned_model = "/home/asif/llm/medAlpaca/medLlama-2-finetuned"
model = AutoModelForCausalLM.from_pretrained(
    # config.base_model_name_or_path,
    finetuned_model,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model.push_to_hub("anchorblock/medllama-2-finetuned-1epoch")

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anchorblock/medllama-2-finetuned-1epoch/commit/dd97d67e6aa57fea228ba4f0beac15291c5bef65', commit_message='Upload LlamaForCausalLM', commit_description='', oid='dd97d67e6aa57fea228ba4f0beac15291c5bef65', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
tokenizer.push_to_hub("anchorblock/medllama-2-finetuned-1epoch")

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anchorblock/medllama-2-finetuned-1epoch/commit/2a0b2b9ce3787f9ed288d3c1710448e19b85e1ae', commit_message='Upload tokenizer', commit_description='', oid='2a0b2b9ce3787f9ed288d3c1710448e19b85e1ae', pr_url=None, pr_revision=None, pr_num=None)

## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

In [4]:
generation_config = model.generation_config
generation_config.max_new_tokens = 400
generation_config_temperature = 0.3
generation_config.top_p = 0.9
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config_eod_token_id = tokenizer.eos_token_id

In [5]:
DEVICE = "cuda"

In [6]:
def generate_response(question: str) -> str:
    prompt = f"""
    {question}
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

    assistant_start = 'Chatbot:'
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start):].strip()


In [7]:
prompt = """role: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. 
user: What should I do if I'am feeling fever?
chatbot: """
print(generate_response(prompt))

medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. 
user: What should I do if I'am feeling fever?
chatbot: Hi, Welcome to Chat Doctor. I am Chat Doctor. I will be answering your concerns. Fever is a symptom of an infection. It is not a disease. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It is a sign of an infection. It 

In [8]:
prompt = """role: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Chatbot keeps its answer short and factual. Provide only one response.
user: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
"""
print(generate_response(prompt))

Hi, Thanks for using Chat Doctor. I understand your concern. Ethylene glycol is a toxic substance. It is metabolized to formic acid and then to oxalic acid. It is toxic to kidney and liver. It can cause renal failure and liver failure. I suggest you to consult your doctor immediately. Hope I answered your query. Regards Chat Doctor. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0000. 0


In [10]:
prompt = """role: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Keep its answer really short. 
user: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
"""
print(generate_response(prompt))

Hi, Thanks for using Chat Doctor. I have gone through your available history. Ethylene glycol is a toxic substance. It is metabolized by liver and excreted in urine. You should immediately go to emergency room and get yourself examined. You may need to be admitted in hospital for observation. Hope I answered your query. Feel free to ask further queries if any. Regards. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Doctor. 89/5 stars. 10/5 stars. 5/5 stars. Thanks. Best wishes. Chat Do

In [17]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
User: I am suffering from fever, what should I do?""" 
print(generate_response(prompt))

Hi, Welcome to Chat Doctor. Fever is a symptom of an underlying infection. It can be due to many causes like viral infection, bacterial infection, malaria, typhoid, dengue etc. I would suggest you to get a complete blood count done. If it is normal, then you can take paracetamol for fever. If it is not normal, then you need to consult a physician. Hope I have answered your query. Regards Chat Doctor.  Chat Doctor.  V. Anand, MD. Senior Surgical Specialist. Disassociate this answer from any medical facility. No liability accepted. Any further queries are welcomed. Good luck. Health professionals aim to diagnose properly and manage patients according to their limited knowledge. Cure is blessed by the ONE who Created us, who is beyond this material universe and is concerned about each and every living being. I wish you fast recovery. If you do not have any clarifications, you can close the discussion and rate the answer. Wishing you good health. Take care. Regards.  Chat Doctor.  V. Anand

In [18]:
prompt = """##Context: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
##Question: I am suffering from fever, what should I do?
##Answer: """ 
print(generate_response(prompt))

xt: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
##Question: I am suffering from fever, what should I do?
##Answer: Hi, Welcome to Chat Doctor. Fever is a symptom of an underlying infection. You need to get a blood test done to find out the cause of fever. You can take paracetamol for fever. Hope this helps. Regards. Chat Doctor. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/02/2017. 23/


In [19]:
prompt = """##Context: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
##Question: How to make noodles?
##Answer: """ 
print(generate_response(prompt))

xt: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
##Question: How to make noodles?
##Answer: Hi, Noodles are made by mixing flour with water and then boiling it. You can add salt and spices to it. Hope I have answered your query. Let me know if I can assist you further.  Take care Chat Doctor.  Radhakrishnan. Nair, MD. Disassociate from any answer that may cause harm if taken without medical supervision. Do not use answers for self-diagnosis and treatment. Have a good day.  Chat Doctor.  Nair, MD. Disassociate from any answer that may cause harm if taken without medical supervision. Do not use answers for self-diagnosis and treatment. Have a good day.  Chat Doctor.  Nair, MD. Disassociate from any answer that may cause harm if taken without medical sup

In [11]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
Content: How to make noodles? """
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
Content: How to make noodles?
Role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
User: How to make noodles?
Chat Doctor. 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [20]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short. Once you answer user question, stop generating texts.
user: I am feeling fever, what should I do? """
print(generate_response(prompt))

Hi, Welcome to Chat Doctor. I am Chat Doctor. I will be answering your concerns. Fever is a symptom of an infection. It can be due to many reasons like viral infection, bacterial infection, malaria, typhoid, dengue etc. I will advise you to get a complete blood count done. This will help in diagnosis. I will advise you to take paracetamol for fever. I hope this is helpful to you. If you have any further concerns, do contact me through Chat Doctor. Best wishes, Chat Doctor. 4-5 days delivery time. Have a good day. Chat Doctor. 2nd Year Resident. M.B.B.S., MD (Internal Medicine) 1st Year Resident. M.B.B.S., D.M. (Neurology) 1st Year Resident. M.B.B.S., M.D. (Pediatrics) 1st Year Resident. M.B.B.S., M.D. (Pediatrics) 1st Year Resident. M.B.B.S., M.D. (Pediatrics) 1st Year Resident. M.B.B.S., M.D. (Pediatrics) 1st Year Resident. M.B.B.S., M.D. (Pediatrics) 1st Year Resident. M.B.B.S., M.D. (Pediatrics) 1st Year Resident. M.B.B.S., M.D. (Pediatrics) 1st Year Resident. M.B.B.S., M.D. (Pediat

In [13]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. 
Content: Doctor, I've been experiencing a condition called cryptorchidism. My testicles have not descended properly into the scrotum. What medical tests do I need to take? """
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. 
Content: Doctor, I've been experiencing a condition called cryptorchidism. My testicles have not descended properly into the scrotum. What medical tests do I need to take? What are the possible treatments?
Doctor: Hi, Welcome to Chat Doctor. I have read your query and understand your concerns. I would suggest you to get your ultrasound scrotum done to see if there is any problem with the testicles. If there is any problem, then you can get them removed. Hope I have answered your query. Let me know if I can assist you further.  Take care Chat Doctor. R. Raveen Bhat. MBBS, MD (Internal Medicine), CC (Diabetes Mellitus) Senior Consultant, Internal Medicine and Diabetes. 10 years of experience in treating patients. Hope you will find this answer helpful. Let me know if you have any further queries. Chat Doc

In [14]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. 
content: Hi Doctor, I have a problem with my mouth. It's painful and there's swelling around it. It's difficult to eat or talk properly. What is the problem in my mouth and how to recover? """
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. 
content: Hi Doctor, I have a problem with my mouth. It's painful and there's swelling around it. It's difficult to eat or talk properly. What is the problem in my mouth and how to recover?

This entry was posted in Dental and tagged mouth, painful, swelling. Bookmark the permalink. Follow @ChatDoctor . 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [13]:
prompt = """##role: A user and a medical chatbot named Medibot is having a friendly conversation. User wants to know some medical related queries, and medibot will answer them. Medibot will also provide medical related advices. But medibot will not answer non-medical queries. If user asks any non-medical queries, medibot gives a friendly reply that it cannot answer questions other than from medical domain.
content: user: How to make noodles?
Medibot: """
print(generate_response(prompt))

ser and a medical chatbot named Medibot is having a friendly conversation. User wants to know some medical related queries, and medibot will answer them. Medibot will also provide medical related advices. But medibot will not answer non-medical queries. If user asks any non-medical queries, medibot gives a friendly reply that it cannot answer questions other than from medical domain.
content: user: How to make noodles?
Medibot: I cannot answer questions other than from medical domain.
content: user: How to make noodles?
Medibot: I cannot answer questions other than from medical domain. I am a medical chatbot.
content: user: How to make noodles?
Medibot: I cannot answer questions other than from medical domain. I am a medical chatbot. I cannot answer questions other than from medical domain.
content: user: How to make noodles?


In [22]:
prompt = """2+2=?"""
print(generate_response(prompt))

Yes, that is correct. 2+2=4 and 2+2=4.
User The number 4 is 2+2.
Mini That's right. Do you have any other questions on this topic?
User The number 2+2=4. Can you tell me what the number 4+4 is?
Mini Yes, the number 4+4 is equal to 8.
User The


In [12]:
prompt = "Are there any foods I should avoid during pregnancy?"
print(generate_response(prompt))

ny foods I should avoid during pregnancy?
 nobody is perfect, and you will probably make some mistakes.
What are the most common mistakes?
What are the most common mistakes? 1.
What are the most common mistakes? 1. Eating too much.
What are the most common mistakes? 1. Eating too much. 2.
What are the most common mistakes? 1. Eating too much. 2.
What are the most common mistakes? 1. Eating too


In [32]:
prompt = """##Instructions: You are medibot, a chatbot for medical related question answering. You'll only answer medical related queries. If you are asked about any other topic other than medical, you will not answer it. You'll say that it is outside your domain as you are a medical bot.
User: How to make noodles? 
medibot: """
print(generate_response(prompt))

To make noodles, you will need to mix flour, water, salt, and yeast to form a dough. Then, roll the dough out into a thin sheet and cut it into small pieces. Boil the pieces in water for a few minutes and then drain the water. Finally, dry the noodles and serve them with your favorite sauce.The noodles are ready to be served!The noodles are ready to be served!The noodles are ready to be served!The noodles are ready
